<a href="https://colab.research.google.com/github/nanmaharaj/FMML_Project_and_Labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [4]:
dataset =  datasets.fetch_california_housing()
#print(dataset.DESCR)  # uncomment this if you want to know more about this dataset(description or metadata about the dataset)
print(dataset.keys())  # if you want to know what else is there in this dataset(dataset os a dictionary)
dataset.target = dataset.target.astype(np.int) # so that we can classify
print(dataset.data.shape)
print(dataset.target.shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])
(20640, 8)
(20640,)


<ipython-input-4-d3bf4ab73e72>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [5]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) #line sums up the squared differences along the first axis (axis 1) of the sq array. This effectively calculates the squared Euclidean distance between the traindata and the query.
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [6]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [7]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

{In Python, `assert` is a keyword that is used as a debugging aid. It allows you to test a condition in your code and trigger an error if the condition is not met. The purpose of using `assert` is to catch and identify bugs and issues during development and testing. Here's how it works:

1. You provide an expression that you expect to be true.

2. If the expression is true, the code continues to execute normally without any issues.

3. If the expression is false, an `AssertionError` exception is raised, which can help you identify where the problem is occurring in your code.

Here's a simple example of how to use `assert`:

```python
x = 10
assert x == 5, "x should be 5"
```

In this example, the `assert` statement checks if `x` is equal to 5. If it's not, an `AssertionError` will be raised with the specified error message "x should be 5."

You can use `assert` for various purposes, such as checking preconditions, postconditions, or invariants in your code, especially during development and testing. However, it's important to note that `assert` statements are typically disabled in production code for performance reasons. You can enable or disable them globally using command-line options or interpreter settings.

Here's how you can run a Python script with assertions enabled:

```bash
python -O script.py
```

The `-O` flag turns off optimizations, which, in turn, enables `assert` statements. In normal execution (without `-O`), `assert` statements are ignored, and no `AssertionError` is raised, regardless of whether the condition is true or false.}

Let us make a function to split the dataset with the desired probability.

In [9]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent #These lines create two boolean arrays, 'split1' and 'split2',
  split1data = data[split1,:] # which indicate whether each sample should belong to the first subset or the second subset based on the randomly generated numbers.
  split2 = rnd>=percent
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [15]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.data),'%')

Number of test samples =  4085
Number of other samples =  16555
Percent of test data =  19.791666666666668 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [17]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [18]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.16961781908500922


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.Assuming that the random number generation is uniform (every class has an equal chance of being selected), the probability of assigning the correct class label to any data point is 1/(number of classes).This is because there are a total of number of classes possible labels, and each label is equally likely to be assigned by random chance.

Let us predict the labels for our validation set and get the accuracy

In [19]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.3480608738340697
Validation accuracy using random classifier is  0.16887579774177713


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [20]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 99.9/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.35


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [21]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.3485924112607099


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answers:
1)
**50-50:** 0.33366178100751326\
**60-40:** 0.33088455772113945\
**65-35:** 0.331940818102698\
**70-30:** 0.339209726443769\
**75-25:** 0.3398034986820034\
**80-20:** 0.3414036163040147\
**85-15:** 0.3450244698205546\
**90-10:** 0.34555288461538464

2)



## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [22]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [23]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.34592423305743425
test accuracy is  0.3485924112607099


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

https://towardsdatascience.com/what-is-cross-validation-60c01f9d9e75

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Answers:

1) On splitting our data in multiple splits and training it, we will obtain more metrics. On observing and averaging these metrics, we get a better idea and a more robust estimation of the accuracy.

2)

Quiz notes:

1)What is the difference between a validation set and a test set?
A) A validation set is used to tune the hyperparameters of a model, while a test set is used to evaluate its performance.
____________________________________________

2)What is the purpose of cross-validation in machine learning?
A) To evaluate the performance of a model on different subsets of the data.

(Cross-validation is a technique for evaluating ML models by training several ML models on subsets of the available input data and evaluating them on the complementary subset of the data. Use cross-validation to detect overfitting, ie, failing to generalize a pattern.)
____________________________________

3)A machine learning problem involves four attributes plus a class. The attributes have 3, 2, 2, and 2 possible values each. The class has 3 possible values. How many maximum possible different examples are there?
A) Maximum possible different examples are the products of the possible values of each attribute and the number of classes;
3 * 2 * 2 * 2 * 3 = 72
___________

4)Which of the following will be the Euclidean distance between two data points A(1, 3) and B(2, 3)?
A) 1

___________________

5)What would be the time taken by 1-NN if there are N observations in the test set? Assume N is very large.
A) N*D

(The time complexity of the KNN algorithm for a single query point is O(nd), where n is the number of training examples and d is the number of features. This is because for each query point, the algorithm needs to compute the distance between the query point and every other point in the dataset.)
